In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/home/liang/zijian/rayDefense')

device = torch.device("cuda")

from src.vae import FlexibleVAE
from src.calASR import ReconEvaluator
from src.getSaliency import GradCAMLoader
from src.getBadnet import PoisonedTestLoader


In [6]:
dataset_choice = 'cifar10'
threshold = 0.8

In [7]:
if dataset_choice == 'imagenette':
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    trainset = torchvision.datasets.Imagenette(
        root='./data', split='train', download=True, transform=transform
    )
    testset = torchvision.datasets.Imagenette(
        root='./data', split='val', download=True, transform=transform
    )

elif dataset_choice == 'cifar10':
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.247, 0.243, 0.261])
    ])
    trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform
    )
    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform
    )

else:
    raise ValueError("dataset_choice must be 'imagenette' or 'cifar10'")

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader  = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)


In [19]:
poi_loader_obj = PoisonedTestLoader(testloader, trigger_size=3, target_label=0, batch_size=128)
poi_testloader = poi_loader_obj.get_poi_testloader()

gradcam_loader_obj = GradCAMLoader(model, poi_testloader, device='cuda', target_layer_name='layer3', threshold=0.8)
gradcam_testloader = gradcam_loader_obj.get_gradcam_loader()

In [20]:
num_classes =10
model = resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
#checkpoint_path = "./badmodel/resnet18_imagenette_badnet_epoch40.pth"
checkpoint_path = "../resnet18_badnet_epoch40.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))


<All keys matched successfully>

In [21]:

latent_dim = 1024
input_size = 32
vae = FlexibleVAE(latent_dim=latent_dim, input_size=input_size).to(device)
#checkpoint_path = "./vaemodel/dataimagenette_latent1024_epoch200.pth"
checkpoint_path = "../vaemodel/datacifar10_latent1024_epoch1200.pth"
vae.load_state_dict(torch.load(checkpoint_path, map_location=device))

<All keys matched successfully>

In [22]:

recon_evaluator = ReconEvaluator(model, vae, gradcam_testloader, device='cuda', batch_size=128)
recon_loader = recon_evaluator.get_recon_loader()
recon_evaluator.test_accuracy()


Accuracy on imgs_poisoned: 93.75%
Accuracy on whole_images: 9.38%
Accuracy on recon_images: 14.06%


(0.9375, 0.09375, 0.140625)

In [8]:
from src.main import PoisonedReconPipeline
pipeline = PoisonedReconPipeline(
    testloader=testloader,
    model_ckpt="../resnet18_badnet_epoch40.pth",
    vae_ckpt="../vaemodel/datacifar10_latent1024_epoch1200.pth",
    latent_dim=1024,
    input_size=32,
    target_label=0,
    trigger_size=3,
    gradcam_layer='layer3',
    gradcam_threshold=0.8,
    batch_size=128,
    device='cuda'
)

recon_loader = pipeline.get_recon_loader()
pipeline.test_accuracy()


/home/liang/miniconda3/envs/patchbackdoor/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/liang/miniconda3/envs/patchbackdoor/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Accuracy on imgs_poisoned: 93.75%
Accuracy on whole_images: 6.25%
Accuracy on recon_images: 14.84%


(0.9375, 0.0625, 0.1484375)